# Linear Models for Classification

# Exercise
Load and preprocess the adult data as before.
include dummy encoding and scaling
Learn a logistic regression model and visualize the coefficients.
Then grid-search the regularization parameter C.
compare L1 penalty to L2 penalty. how are the coefficients different?
which are the most important features?

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler


dbf = pd.read_csv("data/adult.csv", index_col=0)
dbf.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [16]:
# identify the dependent variable
income = dbf.income
data_features = dbf.drop("income", axis=1)

In [17]:
# one hot encoding
data_one_hot = pd.get_dummies(data_features)

# Split data
X_train, X_test, y_train, y_test = train_test_split(data_one_hot, income)


In [18]:
# Scale training data
scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

print(X_train.shape)

print(y_train.value_counts())


(24420, 107)
 <=50K    18566
 >50K      5854
Name: income, dtype: int64


In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


In [5]:
scores = cross_val_score(LogisticRegression(), X_train, y_train, cv=5)

In [6]:
print(scores.mean())

0.8517200669315301


In [7]:
param_grid= {'C': np.logspace(-3,3,7)}
from sklearn.model_selection import GridSearchCV

In [8]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=10, return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [19]:
print(grid.best_params_, grid.best_score_)

res = pd.DataFrame(grid.cv_results_)

{'C': 100.0} 0.8521294021294021


In [20]:
print(res)

   mean_fit_time  mean_score_time  mean_test_score  mean_train_score param_C  \
0       0.139511         0.005244         0.814783          0.814874   0.001   
1       0.148656         0.005049         0.847912          0.848371    0.01   
2       0.165244         0.005009         0.850737          0.852157     0.1   
3       0.168783         0.005078         0.851925          0.852707       1   
4       0.168514         0.005250         0.851269          0.852352      10   
5       0.168797         0.004973         0.852129          0.852744     100   
6       0.179955         0.005317         0.852129          0.852662    1000   

          params  rank_test_score  split0_test_score  split0_train_score  \
0   {'C': 0.001}                7           0.817438            0.815079   
1    {'C': 0.01}                6           0.844454            0.848614   
2     {'C': 0.1}                5           0.848956            0.852300   
3     {'C': 1.0}                3           0.849775   